In [2]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#re-size all the images to this
imageSize = [224, 224]

train_path = 'train'
valid_path = 'test'

In [4]:
# adding preprocessing Layers to the front of vgg
vgg = VGG16(input_shape=imageSize + [3], weights='imagenet',include_top=False)

In [5]:
for layer in vgg.layers:
    layer.trainable=False
x=Flatten()(vgg.output)

In [6]:
prediction=Dense(17,activation='softmax')(x)


In [7]:
model=Model(inputs=vgg.input,outputs=prediction)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
model.compile(
loss='categorical_crossentropy',
    optimizer='adam',
metrics=['accuracy'],run_eagerly=True)

In [10]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [11]:
training_set=train_datagen.flow_from_directory(train_path,
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical')
test_set=test_datagen.flow_from_directory(valid_path,
                                         target_size=(224,224),
                                         batch_size=32,
                                         class_mode='categorical')

Found 655 images belonging to 17 classes.
Found 183 images belonging to 17 classes.


In [12]:
training_set.class_indices

{'Darier_s disease': 0,
 'Muehrck-e_s lines': 1,
 'aloperia areata': 2,
 'beau_s lines': 3,
 'bluish nail': 4,
 'clubbing': 5,
 'eczema': 6,
 'half and half nailes (Lindsay_s nails)': 7,
 'koilonychia': 8,
 'leukonychia': 9,
 'onycholycis': 10,
 'pale nail': 11,
 'red lunula': 12,
 'splinter hemmorrage': 13,
 'terry_s nail': 14,
 'white nail': 15,
 'yellow nails': 16}

In [15]:
data_gen_train = ImageDataGenerator(rescale=1/255.)

data_gen_valid = ImageDataGenerator(rescale=1/255.)

train_generator = data_gen_train.flow_from_directory(train_path, target_size=(224,224), batch_size=32, class_mode="categorical")

valid_generator = data_gen_valid.flow_from_directory(valid_path, target_size=(224,224), batch_size=32, class_mode="categorical")

model.fit(train_generator, epochs=100, validation_data=valid_generator,steps_per_epoch = len(train_generator)//3,validation_steps=len(valid_generator)//3)

Found 655 images belonging to 17 classes.
Found 183 images belonging to 17 classes.
Epoch 1/100
7/7 [==============================] - 234s 30s/step - loss: 3.3060 - accuracy: 0.1401 - val_loss: 3.0256 - val_accuracy: 0.1250
Epoch 2/100
7/7 [==============================] - 184s 27s/step - loss: 2.8714 - accuracy: 0.2232 - val_loss: 2.5144 - val_accuracy: 0.2188
Epoch 3/100
7/7 [==============================] - 169s 24s/step - loss: 2.5026 - accuracy: 0.2802 - val_loss: 2.0834 - val_accuracy: 0.3594
Epoch 4/100
7/7 [==============================] - 213s 31s/step - loss: 2.2788 - accuracy: 0.3527 - val_loss: 2.3035 - val_accuracy: 0.3594
Epoch 5/100
7/7 [==============================] - 190s 27s/step - loss: 1.8377 - accuracy: 0.4464 - val_loss: 1.6384 - val_accuracy: 0.5312
Epoch 6/100
7/7 [==============================] - 168s 24s/step - loss: 1.7040 - accuracy: 0.4928 - val_loss: 1.7214 - val_accuracy: 0.4844
Epoch 7/100
7/7 [==============================] - 156s 22s/step - los

In [16]:
model.save('vgg-16-nail-disease.h5')